In [1]:
import sys
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())
import numpy as np
import scipy
import time
import h5py
import os
from os.path import join
#from configs import configs

import random
#put the path of where main.py runs on the path to simulate that setup for when we actually run
if __name__ == "__main__":
    sys.path.append("../../../../")
from notebooks_src.load_data.label_maker.box_processing.coord_conversion import convert_min_max_to_wh_boxes,_convert_xy_center_wh_box_to_xy_min_max_box
from notebooks_src.load_data.configs import configs
from notebooks_src.plot.plot import plot_boxes
from notebooks_src.load_data.label_maker.box_processing.get_iou import get_iou
%matplotlib inline

importing Jupyter notebook from ../../../../notebooks_src/load_data/label_maker/box_processing/coord_conversion.ipynb
importing Jupyter notebook from ../../../../notebooks_src/load_data/configs.ipynb
importing Jupyter notebook from ../../../../notebooks_src/plot/plot.ipynb
importing Jupyter notebook from ../../../../notebooks_src/load_data/label_maker/box_processing/get_iou.ipynb


In [2]:
def create_labelmaker_fxn(all_labels):
    '''all_labels is a list of box_lists, where each box is a 5-D vector '''
    pass

def get_all_default_boxes(all_labels):
    pass

In [3]:
def create_default_box_shapes_for_feature_map(fmap_receptive_field_size, filter_size):
    rf = fmap_receptive_field_size
    """takes in
           int: receptive_field_size (1 for orginal image, 2 for downsampled by 2 image,... n for downsampled by n image )
           int: filter_size: size of filter (we assume filters are square) 
        returns list of 2d tuples which contain (width, height) of box"""
    """by default lets do a (rf = receptive field size):
            * squares: rf x rf box, 2rf x 2rf,..., (filter_size -1) * rf x (filter_size -1)*rf box
            * rectangle rf x 2rf, 2rf x rf,..., (filter_size -1) * rf x rf, rf x (filter_size -1) * rf
        so the larger the receptive field -> the larger the context for the box aka the larger the filter_size * rf to box size ratio
     """
    default_boxes = []
    for scale in range(1,filter_size):
        sq_boxes = make_square_boxes(scale, rf)
        default_boxes.extend(sq_boxes)
        if scale != 1:
            rect_boxes = make_rectangle_boxes(scale, rf)
            default_boxes.extend(rect_boxes)
    #get rid of duplicated
    default_boxes = list(set(default_boxes))
    return default_boxes
        
        
        

def make_square_boxes(scale, receptive_field_size):
    rf = receptive_field_size
    return [(scale*rf, scale*rf)]

def make_rectangle_boxes(scale, receptive_field_size):
    rf = receptive_field_size
    rect_boxes = []
    for mult in range(scale, 0, -1):
        rect_boxes.append(( scale * rf, mult * rf))
        rect_boxes.append(( mult * rf, scale *rf, ))
    return rect_boxes

In [9]:
def create_default_boxes_for_feature_map(box_lists, feature_map_size, receptive_field_size, filter_size, use_cutoff_boxes=False):
    
    """ takes in:
                 feature map size (tuple): (xdim,ydim)
                 receptive field size (int)
                 filter_size (int): we assume square filters for now
                 use_cutoff_boxes (bool): whether to have boxes that get cut off from image"""
    boxes = []
    box_shapes = create_default_box_shapes_for_feature_map(receptive_field_size, filter_size)
    num_channels_for_one_box = 4 + 1
    xdim, ydim = feature_map_size
    
    gt_box_tensor = make_empty_box_tensor(box_shapes, box_lists, xdim,ydim)
    
    for example_ind in range(box_lists.shape[0]):
        gt_boxes = box_lists[example_ind]
        for xind in range(xdim):
            for yind in range(ydim):
                for def_box_ind, box_shape in enumerate(box_shapes):
                    channel_ind = num_channels_for_one_box * def_box_ind
                    xcenter, ycenter = (xind + 0.5) * receptive_field_size, (yind + 0.5) * receptive_field_size
                    width, height = box_shape
                    xmin, xmax, ymin, ymax = _convert_xy_center_wh_box_to_xy_min_max_box(xcenter, ycenter, width, height)
                    if not use_cutoff_boxes:
                        if does_box_get_cutoff(xmin, ymin, xmax, ymax,
                                            frame_xmax=xdim*receptive_field_size, 
                                            frame_ymax=ydim*receptive_field_size):
                            continue

                    default_box = (xcenter, ycenter, width, height)
                    is_match, xoff,yoff, scaled_w, scaled_h, cls = match_boxes(default_box, gt_boxes)
                    
                    if is_match:
                        # set the correct default box location to be offset from that default box
                        gt_box_tensor[example_ind, channel_ind:channel_ind + 4, xind, yind] = [xoff,yoff, scaled_w, scaled_h]
                        
                        # set class index to class number for categorical encoding
                        gt_box_tensor[example_ind, channel_ind:channel_ind + num_channels_for_one_box, xind, yind] = cls
                
    return gt_box_tensor
            
    

In [5]:
def make_empty_box_tensor(box_shapes, box_lists, xdim,ydim):
    num_default_boxes_per_cell = len(box_shapes)
    num_examples = len(box_lists)
    num_channels_in_gt_box_tensor = num_channels_for_one_box * num_default_boxes_per_cell
    gt_box_tensor = np.zeros(num_examples, num_channels_in_gt_box_tensor, xdim, ydim)
    return gt_box_tensor
    

In [ ]:
def does_box_get_cutoff(xmin,ymin,xmax,ymax, frame_xmax, frame_ymax):
    is_cutoff = False
    if xmin < 0 and ymin < 0:
        if xmax > frame_xmax and ymax > frame_ymax:
            is_cutoff=True
    return is_cutoff

In [2]:
def match_boxes(default_box, gt_boxes, thresh=0.5):
    #is gt_boxes xywh or xmin,xmax,...?
    xoff,yoff, scaled_w, scaled_h, cls = 5*[0.]
    ious = [get_iou(default_box, gt_box) for gt_box in gt_boxes]
    max_iou = np.max(ious)
    if max_iou >= thresh:
        max_iou_ind = np.argmax(ious)
        best_gt = gt_boxes[max_iou_ind]
        xoff, yoff = calc_xyoff(default_box, best_gt)
        scaled_w, scaled_h = calc_wh_scale(default_box, best_gt)
        is_match = True
        
    else:
        is_match = False
    return is_match, xoff,yoff, scaled_w, scaled_h, cls
        
    
    

In [3]:
def calc_xyoff(default_box, best_gt):
    dx, dy = default_box[:2]
    gx, gy = best_gt[:2]
    xoff = (gx - dx) / float(dx)
    yoff = (gy - dy) / float(dy)
    return xoff,yoff
    

In [4]:
def calc_wh_scale(default_box, best_gt):
    dw, dh = default_box[2:4]
    gw, gh = best_gt[2:4]
    scaled_w  = np.log2(float(gw) / dw)
    scaled_h  = np.log2(float(gh) / dh)
    return scaled_w, scaled_h
    